# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [102]:
import pandas as pd

## imports for Python, Pandas

In [103]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{'_id': {'$oid': '52b213b38594d8a2be17c780'},
  'approvalfy': 1999,
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': 0,
  'ibrdcommamt': 0,
  'id': 'P129828',
  'idacommamt': 130000000,
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': 550000000,
  'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
   {'Name': 'Education', 'Percent': 26},
   {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
   {'Name': 'Educatio

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df
#print(type(sample_json_df))

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## Problem #1: Find the 10 countries with the most projects

In [3]:
#load as pandas dataframe
df = pd.read_json('data/world_bank_projects.json')
  #pandas.core.frame.DataFrame - shows this json is a pandas dataFrame and not dict

In [104]:
#Let's do some index searching to make sure we can explore..hmmmm looks like a dictionary, but its not!
#https://dsintensivecommunity.springboard.com/t/m2tm03/unit-3-3-json-exercise-n3-lookup-table
df['_id'][3]
with open('data/world_bank_projects.json') as f:
    raw = json.load(f)

In [5]:
#Use describe to look at the interesting details
df.countrycode.describe()

count     500
unique    118
top        CN
freq       19
Name: countrycode, dtype: object

#### Interesting, there are 500 rows, but only 118 unique countries

In [6]:
# Let's use groupby to aggregate the count of each country. First, extract the 'countryname' column from the DataFrame.
by_country = df.groupby('countryname')
#Next, aggregate 'countryname' and count the totals. This will not place it in sorted order.
count_by_country = by_country['countryname'].count()
#Finally, sort so can see the totals by country. 
count_by_country.sort_values(ascending=False).head(10)
#to reuse the output, can assign to a variable to select only the top 10. 
#Note: To see all of the countries, remove .head(10) and run the below line:
#print(count_by_country.sort_values(ascending=False))

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

# Answer: above are the top 10 countries with the most projects

# Question 2.Find the top 10 major project themes (using column 'mjtheme_namecode')

In [7]:
#Find the column names. 
df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')

In [107]:
print(df['mjtheme_namecode'])

0      [{'code': '8', 'name': 'Human development'}, {...
1      [{'code': '1', 'name': 'Economic management'},...
2      [{'code': '5', 'name': 'Trade and integration'...
3      [{'code': '7', 'name': 'Social dev/gender/incl...
4      [{'code': '5', 'name': 'Trade and integration'...
5      [{'code': '6', 'name': 'Social protection and ...
6      [{'code': '2', 'name': 'Public sector governan...
7      [{'code': '11', 'name': 'Environment and natur...
8      [{'code': '10', 'name': 'Rural development'}, ...
9      [{'code': '2', 'name': 'Public sector governan...
10     [{'code': '10', 'name': 'Rural development'}, ...
11     [{'code': '10', 'name': 'Rural development'}, ...
12     [{'code': '4', 'name': 'Financial and private ...
13     [{'code': '5', 'name': 'Trade and integration'...
14     [{'code': '6', 'name': 'Social protection and ...
15     [{'code': '10', 'name': 'Rural development'}, ...
16     [{'code': '10', 'name': 'Rural development'}, ...
17     [{'code': '8', 'name': '

In [9]:
#test - cycle through and print the key value pairs. The index integer is the key and list of lists is the value
major_theme = df['mjtheme_namecode']
for k,v in major_theme.items():
    print(k,":\"<-key\" ",v)
print(type(major_theme)) #pandas series

0 :"<-key"  [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
1 :"<-key"  [{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]
2 :"<-key"  [{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
3 :"<-key"  [{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]
4 :"<-key"  [{'code': '5', 'name': 'Trade and integration'}, {'code': '4', 'name': 'Financial and private sector development'}]
5 :"<-key"  [{'code': '6', 'name': 'Social protection and risk management'}, {'code': '6', 'name': ''}]
6 :"<-key"  [{'code': '2', 'name': 'Public sector governance'}, {'code': '4', 'name': 'Financial and private sector development'}]
7 :"<-key"  [{'code': '11', 'name': 'Environment and natural

In [10]:
major_theme[11][3]

{'code': '11', 'name': 'Environment and natural resources management'}

In [11]:
#Load json as a list. 
data2 = json.load(open('data/world_bank_projects.json'))
type(data2)

list

In [108]:
#I have to normalize this  because 'mjtheme_namecode' is embedded. This makes norm a pandas DataFrame
norm = json_normalize(data2,'mjtheme_namecode',['id'])
print(type(norm))
print(norm)
#each row is made up of a code and name. See below to group by distinct codes/names

<class 'pandas.core.frame.DataFrame'>
     code                                          name       id
0       8                             Human development  P129828
1      11                                                P129828
2       1                           Economic management  P144674
3       6         Social protection and risk management  P144674
4       5                         Trade and integration  P145310
5       2                      Public sector governance  P145310
6      11  Environment and natural resources management  P145310
7       6         Social protection and risk management  P145310
8       7                   Social dev/gender/inclusion  P144665
9       7                   Social dev/gender/inclusion  P144665
10      5                         Trade and integration  P144933
11      4      Financial and private sector development  P144933
12      6         Social protection and risk management  P146161
13      6                                           

In [109]:
#Group by the name of the mjtheme_namecode column
major_theme_names = norm.groupby('code')['name'].unique()
print(major_theme_names)
#You will see 11 distinct codes, and 12 distinct names (this is because the 'name' is blank for the codes at 
#the top of the below list)

code
1                              [Economic management, ]
10                               [Rural development, ]
11    [, Environment and natural resources management]
2                         [Public sector governance, ]
3                                      [Rule of law, ]
4         [Financial and private sector development, ]
5                            [Trade and integration, ]
6            [Social protection and risk management, ]
7                      [Social dev/gender/inclusion, ]
8                                [Human development, ]
9                                [Urban development, ]
Name: name, dtype: object


In [115]:
#c1 = norm.groupby('code')['name'].count()     #this counts the codes, since some codes have a null name, using the below groupby
#would cause the list to miss these distinct codes.
c1 = norm.groupby(norm['code'], as_index=True, sort=True,group_keys=True).size()
#c2 = c1.set_index(['name'])
#from: https://stackoverflow.com/questions/34113203/python-pandas-setting-groupby-group-labels-as-index-in-a-new-dataframe
#c1 = df.groupby(df['Country'], as_index=True, sort=False, group_keys=True).size()
#c1.set_index(['Country'])
c1
#norm.groupby('name')['code'].count()

code
1      38
10    216
11    250
2     199
3      15
4     146
5      77
6     168
7     130
8     210
9      50
dtype: int64

# Answer #2: See above

# Question 3: In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [119]:
#norm.groupby('id').code.nunique().sort_values(ascending=False).head(10)
norm.name.value_counts(ascending=False)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64

In [121]:
#create lookup table from code to name:
df_themes_name_to_code = norm.groupby('name').code.max().sort_values(ascending=False)

# drop the missing name rows
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != '']

In [124]:
# convert to df
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code,columns=['code'])

# pull name into a column
df_themes_name_to_code['name_clean'] = df_themes_name_to_code.index

# set code to be the index
df_themes_code_to_name = df_themes_name_to_code.set_index(['code'])

In [127]:
print(norm.shape)
df_themes_clean = norm.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)
print(df_themes_clean.shape)
print(df_themes_clean)

(1499, 3)
(1499, 4)
     code               name       id         name_clean
0       8  Human development  P129828  Human development
17      8                     P127033  Human development
41      8  Human development  P132616  Human development
43      8  Human development  P146271  Human development
44      8  Human development  P146271  Human development
47      8  Human development  P146271  Human development
57      8  Human development  P146109  Human development
58      8  Human development  P146109  Human development
69      8  Human development  P143025  Human development
79      8  Human development  P122700  Human development
91      8  Human development  P145118  Human development
113     8  Human development  P128628  Human development
117     8                     P098847  Human development
120     8  Human development  P125018  Human development
146     8  Human development  P132753  Human development
147     8  Human development  P132753  Human development
148     8  

In [88]:
#create function that can drop in correct names for normalized json column 'mjtheme_namecode
def clean_mjtheme(value):
    for name in value:
        if len(name[0]) < 1:
            name[1]
        else:
            name[0]
    return value

output = clean_mjtheme(norm) 
output

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [101]:
#replace '' with the 'name' value associated with the correct 'code'. See here for functions with lambda and the .apply method:
#https://campus.datacamp.com/courses/cleaning-data-in-python/cleaning-data-for-analysis?ex=10 
#df['mjtheme_namecode'].apply(lambda x)
def filled_name(value):
    for clean in value:
        print([{'code': clean['code'], 
                'name' : output.loc[clean['code']]}])      # clean_mjtheme.loc(norm)
        return value
filled_name(output)

KeyError: 'the label [c] is not in the [index]'